# Automated ML

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.widgets import RunDetails

import joblib
from azureml.core.model import Model

from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice

from train import clean_data

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-gridstability-automl'

experiment=Experiment(ws, experiment_name)

In [ ]:
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
run = experiment.start_logging()

In [ ]:
cpu_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster already exists. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ds = pd.read_csv("data/gridstability.csv")

In [ ]:
x, y = clean_data(ds)

In [ ]:
ds_clean= pd.concat([x, y], axis=1)
ds_clean.to_csv("data/ds_clean.csv", index=False)

dataset = Dataset.Tabular.from_delimited_files("data/ds_clean.csv")

In [ ]:
dataset.take(5).to_pandas_dataframe()

In [ ]:
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
project_folder = 'gridstability-runs'

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'Accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=ds,
                             label_column_name="stabf",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
metrics_output = remote_run.get_pipeline_output('metrics_output')
num_file_downloaded = metrics_output.download('.', show_progress=True)

In [ ]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

In [ ]:
best_run, model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

In [ ]:
best_run_metrics['accuracy']

In [ ]:
model

In [ ]:
joblib.dump(model, filename="outputs/best_model_automl.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
remote_run.register_model(model_name = 'model_automl', model_path='outputs/model_automl.pkl')

In [ ]:
environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)

inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

In [ ]:
%run endpoint.py

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()